In [1]:
cd

/home/ubuntu


In [2]:
from DemoServerDB import ConnectDatabase
from mapping import doc

In [3]:
temp = ConnectDatabase()

In [4]:
temp.conn

<connection object at 0x7f16f5f77b40; dsn: 'dbname=postgresdemo user=clientdemo password=xxxxxxxxxxx host=postgresdemo.c3yxphqgag3s.ap-southeast-1.rds.amazonaws.com port=5432 sslmode=require', closed: 0>

In [5]:
doc['column_map']['TRANSACTION_MASTER']

{'cust_id': 'userid',
 'product_id': 'product_id',
 'quantity': 'quantity',
 'renewal': 'renewal',
 'revenue': 'revenue',
 'timestamp': 'purchase_date'}

In [6]:
query = 'SELECT "%s","%s","%s" FROM "%s"'%(doc['column_map']['TRANSACTION_MASTER']['cust_id'],
                                           doc['column_map']['TRANSACTION_MASTER']['product_id'],
                                           doc['column_map']['TRANSACTION_MASTER']['timestamp'],
                                           doc['table_map']['TRANSACTION_MASTER'])

In [7]:
print query

SELECT "userid","product_id","purchase_date" FROM "trans"


In [8]:
temp.conn.rollback()

In [9]:
temp.cur.execute(query)

In [10]:
col_names = [desc[0] for desc in temp.cur.description]

In [11]:
col_names

['userid', 'product_id', 'purchase_date']

In [12]:
import pandas as pd
import numpy as np

In [13]:
trans_table = pd.DataFrame(temp.cur.fetchall(), columns=col_names)

In [14]:
trans_table.head()

,userid,product_id,purchase_date
0,3893506,com.gopaktor.subscription.v1.premium.1m,2016-09-07 13:59:45+00:00
1,3793254,com.gopaktor.subscription.v1.premium.1m,2016-09-25 03:06:11+00:00
2,6148794,com.gopaktor.subscription.v1.premium.1m,2016-10-01 02:27:37+00:00
3,6149220,com.gopaktor.subscription.v4.premium.1m,2016-10-01 06:39:16+00:00
4,6149432,com.gopaktor.subscription.v4.premium.1m,2016-10-01 09:07:49+00:00


In [15]:
trans_table.rename(columns={doc['column_map']['TRANSACTION_MASTER']['cust_id']:'cust_id'},inplace=True)
trans_table.rename(columns={doc['column_map']['TRANSACTION_MASTER']['product_id']:'product_id'},inplace=True)
trans_table.rename(columns={doc['column_map']['TRANSACTION_MASTER']['timestamp']:'timestamp'},inplace=True)

In [16]:
trans_table.head()

,cust_id,product_id,timestamp
0,3893506,com.gopaktor.subscription.v1.premium.1m,2016-09-07 13:59:45+00:00
1,3793254,com.gopaktor.subscription.v1.premium.1m,2016-09-25 03:06:11+00:00
2,6148794,com.gopaktor.subscription.v1.premium.1m,2016-10-01 02:27:37+00:00
3,6149220,com.gopaktor.subscription.v4.premium.1m,2016-10-01 06:39:16+00:00
4,6149432,com.gopaktor.subscription.v4.premium.1m,2016-10-01 09:07:49+00:00


In [17]:
cust_prod = trans_table[['cust_id', 'product_id']]

In [18]:
grp_cust = cust_prod.groupby('cust_id').count().reset_index()

In [19]:
grp_cust.head()

,cust_id,product_id
0,2942914,1
1,3053432,10
2,3129800,1
3,3289765,1
4,3325662,2


In [20]:
grp_cust.rename(columns={'product_id':'count'}, inplace=True)

In [21]:
grp_cust.head()

,cust_id,count
0,2942914,1
1,3053432,10
2,3129800,1
3,3289765,1
4,3325662,2


In [ ]:
grp_cust.shape

In [22]:
group_data = trans_table.groupby(['cust_id', 'product_id']).size().reset_index().rename(columns={0:'count'})

In [23]:
group_data.head()

,cust_id,product_id,count
0,2942914,com.gopaktor.subscription.v1.premium.1m,1
1,3053432,com.gopaktor.subscription.v4.premium.1m,10
2,3129800,com.gopaktor.subscription.v1.premium.1m,1
3,3289765,com.gopaktor.subscription.v2.premium.6m.2000,1
4,3325662,com.gopaktor.subscription.v12.premium.1m,2


In [24]:
group_data.shape

(3232, 3)

In [25]:
repeated_data = group_data.query('count != 1')

In [26]:
repeated_data.head()

,cust_id,product_id,count
1,3053432,com.gopaktor.subscription.v4.premium.1m,10
4,3325662,com.gopaktor.subscription.v12.premium.1m,2
5,3409109,com.gopaktor.subscription.v12.premium.1m,2
6,3793254,com.gopaktor.subscription.v1.premium.1m,4
7,3893506,com.gopaktor.subscription.v1.premium.1m,4


In [27]:
len(repeated_data.cust_id.unique())

796

In [28]:
len(repeated_data.product_id.unique())

9

In [29]:
repeated_data.head()

,cust_id,product_id,count
1,3053432,com.gopaktor.subscription.v4.premium.1m,10
4,3325662,com.gopaktor.subscription.v12.premium.1m,2
5,3409109,com.gopaktor.subscription.v12.premium.1m,2
6,3793254,com.gopaktor.subscription.v1.premium.1m,4
7,3893506,com.gopaktor.subscription.v1.premium.1m,4


In [30]:
trans_table.head()

,cust_id,product_id,timestamp
0,3893506,com.gopaktor.subscription.v1.premium.1m,2016-09-07 13:59:45+00:00
1,3793254,com.gopaktor.subscription.v1.premium.1m,2016-09-25 03:06:11+00:00
2,6148794,com.gopaktor.subscription.v1.premium.1m,2016-10-01 02:27:37+00:00
3,6149220,com.gopaktor.subscription.v4.premium.1m,2016-10-01 06:39:16+00:00
4,6149432,com.gopaktor.subscription.v4.premium.1m,2016-10-01 09:07:49+00:00


In [31]:
xyz = pd.merge(trans_table, repeated_data, on=['cust_id', 'product_id'], how='left')

In [32]:
xyz.head()

,cust_id,product_id,timestamp,count
0,3893506,com.gopaktor.subscription.v1.premium.1m,2016-09-07 13:59:45+00:00,4.0
1,3793254,com.gopaktor.subscription.v1.premium.1m,2016-09-25 03:06:11+00:00,4.0
2,6148794,com.gopaktor.subscription.v1.premium.1m,2016-10-01 02:27:37+00:00,NaN
3,6149220,com.gopaktor.subscription.v4.premium.1m,2016-10-01 06:39:16+00:00,NaN
4,6149432,com.gopaktor.subscription.v4.premium.1m,2016-10-01 09:07:49+00:00,NaN


In [33]:
xyz.shape

(4428, 4)

In [34]:
len(xyz.cust_id.unique())

3087

In [35]:
len(xyz.product_id.unique())

18

In [51]:
def x(frame):
    frame['time'] = frame['timestamp'].max() - frame['timestamp'].min()
    return frame

In [52]:
dates = xyz.groupby(['cust_id','product_id'], group_keys=False).apply(x)

In [53]:
dates.head()

,cust_id,product_id,timestamp,count,time
0,3893506,com.gopaktor.subscription.v1.premium.1m,2016-09-07 13:59:45+00:00,4.0,91 days 01:00:00
1,3793254,com.gopaktor.subscription.v1.premium.1m,2016-09-25 03:06:11+00:00,4.0,91 days 01:00:00
2,6148794,com.gopaktor.subscription.v1.premium.1m,2016-10-01 02:27:37+00:00,NaN,0 days 00:00:00
3,6149220,com.gopaktor.subscription.v4.premium.1m,2016-10-01 06:39:16+00:00,NaN,0 days 00:00:00
4,6149432,com.gopaktor.subscription.v4.premium.1m,2016-10-01 09:07:49+00:00,NaN,0 days 00:00:00


In [55]:
cust_repur = dates

In [57]:
cust_repur.head()

,cust_id,product_id,timestamp,count,time
0,3893506,com.gopaktor.subscription.v1.premium.1m,2016-09-07 13:59:45+00:00,4.0,91 days 01:00:00
1,3793254,com.gopaktor.subscription.v1.premium.1m,2016-09-25 03:06:11+00:00,4.0,91 days 01:00:00
2,6148794,com.gopaktor.subscription.v1.premium.1m,2016-10-01 02:27:37+00:00,NaN,0 days 00:00:00
3,6149220,com.gopaktor.subscription.v4.premium.1m,2016-10-01 06:39:16+00:00,NaN,0 days 00:00:00
4,6149432,com.gopaktor.subscription.v4.premium.1m,2016-10-01 09:07:49+00:00,NaN,0 days 00:00:00


In [58]:
cust_repur.shape

(4428, 5)

In [59]:
len(cust_repur.product_id.unique())

18

In [60]:
cust_repur.reset_index(drop=True, inplace=True)

In [61]:
cust_repur.head()

,cust_id,product_id,timestamp,count,time
0,3893506,com.gopaktor.subscription.v1.premium.1m,2016-09-07 13:59:45+00:00,4.0,91 days 01:00:00
1,3793254,com.gopaktor.subscription.v1.premium.1m,2016-09-25 03:06:11+00:00,4.0,91 days 01:00:00
2,6148794,com.gopaktor.subscription.v1.premium.1m,2016-10-01 02:27:37+00:00,NaN,0 days 00:00:00
3,6149220,com.gopaktor.subscription.v4.premium.1m,2016-10-01 06:39:16+00:00,NaN,0 days 00:00:00
4,6149432,com.gopaktor.subscription.v4.premium.1m,2016-10-01 09:07:49+00:00,NaN,0 days 00:00:00


In [130]:
kkkk = cust_repur.groupby(['cust_id', 'product_id'])['timestamp'].max()

In [132]:
kk = kkkk.to_frame().reset_index()

In [133]:
kk.head()

,cust_id,product_id,timestamp
0,2942914,com.gopaktor.subscription.v1.premium.1m,2016-12-24 09:53:35+00:00
1,3053432,com.gopaktor.subscription.v4.premium.1m,2017-01-26 06:22:00+00:00
2,3129800,com.gopaktor.subscription.v1.premium.1m,2016-12-09 07:57:37+00:00
3,3289765,com.gopaktor.subscription.v2.premium.6m.2000,2016-12-06 11:37:37+00:00
4,3325662,com.gopaktor.subscription.v12.premium.1m,2016-12-03 04:36:00+00:00


In [126]:
kk.shape

(3232, 3)

In [62]:
list_of_indexes = [np.argmax(g['timestamp']) for l, g in cust_repur.groupby(['cust_id','product_id'])]

In [179]:
cust_repur_original = cust_repur.ix[list_of_indexes]

In [180]:
cust_repur_original.head()

,cust_id,product_id,timestamp,count,time
2790,2942914,com.gopaktor.subscription.v1.premium.1m,2016-12-24 09:53:35+00:00,NaN,0 days 00:00:00
3556,3053432,com.gopaktor.subscription.v4.premium.1m,2017-01-26 06:22:00+00:00,10.0,245 days 01:54:56
2529,3129800,com.gopaktor.subscription.v1.premium.1m,2016-12-09 07:57:37+00:00,NaN,0 days 00:00:00
1308,3289765,com.gopaktor.subscription.v2.premium.6m.2000,2016-12-06 11:37:37+00:00,NaN,0 days 00:00:00
2411,3325662,com.gopaktor.subscription.v12.premium.1m,2016-12-03 04:36:00+00:00,2.0,31 days 01:49:48


In [183]:
cust_repur_original["timestamp"].dt.date

2790    2016-12-24
3556    2017-01-26
2529    2016-12-09
1308    2016-12-06
2411    2016-12-03
3697    2016-12-30
1949    2016-12-25
2504    2016-12-07
2577    2016-12-12
868     2016-11-15
2       2016-10-01
3264    2017-01-17
1732    2016-11-05
213     2016-10-01
2196    2016-11-22
3       2016-10-01
17      2016-10-02
3550    2017-02-01
556     2016-11-02
58      2016-10-07
1603    2016-11-02
4       2016-10-01
5       2016-10-01
1861    2016-11-09
1602    2016-11-02
2019    2016-11-14
237     2016-10-04
1245    2016-12-06
8       2016-10-01
1285    2016-12-08
           ...    
4386    2017-02-03
4391    2017-02-03
3627    2017-02-03
4390    2017-02-03
3634    2017-02-04
4395    2017-02-04
3638    2017-02-04
4397    2017-02-04
3656    2017-02-04
4398    2017-02-04
3644    2017-02-04
3646    2017-02-04
3648    2017-02-04
3650    2017-02-04
4405    2017-02-04
4410    2017-02-05
4412    2017-02-05
4416    2017-02-05
3658    2017-02-05
3659    2017-02-05
3661    2017-02-05
3663    2017

In [108]:
cust_repur_original.shape

(3232, 8)

In [65]:
cust_repur_original.reset_index(drop=True,inplace=True)

In [66]:
cust_repur_original.head()

,cust_id,product_id,timestamp,count,time
0,2942914,com.gopaktor.subscription.v1.premium.1m,2016-12-24 09:53:35+00:00,NaN,0 days 00:00:00
1,3053432,com.gopaktor.subscription.v4.premium.1m,2017-01-26 06:22:00+00:00,10.0,245 days 01:54:56
2,3129800,com.gopaktor.subscription.v1.premium.1m,2016-12-09 07:57:37+00:00,NaN,0 days 00:00:00
3,3289765,com.gopaktor.subscription.v2.premium.6m.2000,2016-12-06 11:37:37+00:00,NaN,0 days 00:00:00
4,3325662,com.gopaktor.subscription.v12.premium.1m,2016-12-03 04:36:00+00:00,2.0,31 days 01:49:48


In [69]:
cust_repur_original['days_int'] = ""

In [143]:
cust_repur_original["timestamp"] = pd.to_datetime(cust_repur_original["timestamp"]) 

In [150]:
cust_repur_original["timestamp"].dt

In [100]:
import datetime as dt

In [101]:
cust_repur_original['xyz'] = cust_repur_original['time'].dt.days

In [102]:
cust_repur_original.head()

,cust_id,product_id,timestamp,count,time,days_int,repurchase_time,xyz
0,2942914,com.gopaktor.subscription.v1.premium.1m,2016-12-24 09:53:35+00:00,NaN,0 days 00:00:00,0,NaN,0
1,3053432,com.gopaktor.subscription.v4.premium.1m,2017-01-26 06:22:00+00:00,10.0,245 days 01:54:56,245,27.1111,245
2,3129800,com.gopaktor.subscription.v1.premium.1m,2016-12-09 07:57:37+00:00,NaN,0 days 00:00:00,0,NaN,0
3,3289765,com.gopaktor.subscription.v2.premium.6m.2000,2016-12-06 11:37:37+00:00,NaN,0 days 00:00:00,0,NaN,0
4,3325662,com.gopaktor.subscription.v12.premium.1m,2016-12-03 04:36:00+00:00,2.0,31 days 01:49:48,31,30,31


In [75]:
for index, row in cust_repur_original.iterrows():
    cust_repur_original.days_int[index] = cust_repur_original.time[index].days

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [76]:
cust_repur_original.head()

,cust_id,product_id,timestamp,count,time,days_int
0,2942914,com.gopaktor.subscription.v1.premium.1m,2016-12-24 09:53:35+00:00,NaN,0 days 00:00:00,0
1,3053432,com.gopaktor.subscription.v4.premium.1m,2017-01-26 06:22:00+00:00,10.0,245 days 01:54:56,245
2,3129800,com.gopaktor.subscription.v1.premium.1m,2016-12-09 07:57:37+00:00,NaN,0 days 00:00:00,0
3,3289765,com.gopaktor.subscription.v2.premium.6m.2000,2016-12-06 11:37:37+00:00,NaN,0 days 00:00:00,0
4,3325662,com.gopaktor.subscription.v12.premium.1m,2016-12-03 04:36:00+00:00,2.0,31 days 01:49:48,31


In [ ]:
cust_repur_original.head()

In [207]:
cust_repur_original['repurchase_time'] = (cust_repur_original['days_int']-1) / (cust_repur_original['count']-1)

KeyError: 'days_int'

In [80]:
cust_repur_original.head()

,cust_id,product_id,timestamp,count,time,days_int,repurchase_time
0,2942914,com.gopaktor.subscription.v1.premium.1m,2016-12-24 09:53:35+00:00,NaN,0 days 00:00:00,0,NaN
1,3053432,com.gopaktor.subscription.v4.premium.1m,2017-01-26 06:22:00+00:00,10.0,245 days 01:54:56,245,27.1111
2,3129800,com.gopaktor.subscription.v1.premium.1m,2016-12-09 07:57:37+00:00,NaN,0 days 00:00:00,0,NaN
3,3289765,com.gopaktor.subscription.v2.premium.6m.2000,2016-12-06 11:37:37+00:00,NaN,0 days 00:00:00,0,NaN
4,3325662,com.gopaktor.subscription.v12.premium.1m,2016-12-03 04:36:00+00:00,2.0,31 days 01:49:48,31,30


In [81]:
cust_repur_original.head()

,cust_id,product_id,timestamp,count,time,days_int,repurchase_time
0,2942914,com.gopaktor.subscription.v1.premium.1m,2016-12-24 09:53:35+00:00,NaN,0 days 00:00:00,0,NaN
1,3053432,com.gopaktor.subscription.v4.premium.1m,2017-01-26 06:22:00+00:00,10.0,245 days 01:54:56,245,27.1111
2,3129800,com.gopaktor.subscription.v1.premium.1m,2016-12-09 07:57:37+00:00,NaN,0 days 00:00:00,0,NaN
3,3289765,com.gopaktor.subscription.v2.premium.6m.2000,2016-12-06 11:37:37+00:00,NaN,0 days 00:00:00,0,NaN
4,3325662,com.gopaktor.subscription.v12.premium.1m,2016-12-03 04:36:00+00:00,2.0,31 days 01:49:48,31,30


In [ ]:
len(cust_repur_original.product_id.unique())

In [82]:
cust_repur_original.product_id.unique()

array(['com.gopaktor.subscription.v1.premium.1m',
       'com.gopaktor.subscription.v4.premium.1m',
       'com.gopaktor.subscription.v2.premium.6m.2000',
       'com.gopaktor.subscription.v12.premium.1m',
       'com.gopaktor.subscription.v2.premium.1m.2000',
       'com.gopaktor.credit.v2.1000',
       'com.gopaktor.subscription.v2.premium.3m.2000',
       'com.gopaktor.subscription.v3.premium.6m',
       'com.gopaktor.subscription.v3.premium.3m',
       'com.gopaktor.subscription.v12.premium.3m',
       'com.gopaktor.subscription.v2.premium.3m',
       'com.gopaktor.subscription.v2.premium.6m',
       'com.gopaktor.subscription.v2.premium.1m.3000',
       'com.gopaktor.subscription.v11.premium.6m',
       'com.gopaktor.credit.v2.3000', 'com.gopaktor.credit.v2.10000',
       'com.gopaktor.credit.v2.6000', 'com.gopaktor.credit.v3.2000'], dtype=object)

In [84]:
abc = pd.pivot_table(cust_repur_original, index=['cust_id'], columns=['product_id'], values=['repurchase_time'], aggfunc='first')

In [85]:
abc.shape

(3087, 18)

In [86]:
abc.head()

repurchase_time                               \
product_id com.gopaktor.credit.v2.1000 com.gopaktor.credit.v2.10000   
cust_id                                                               
2942914                            NaN                          NaN   
3053432                            NaN                          NaN   
3129800                            NaN                          NaN   
3289765                            NaN                          NaN   
3325662                            NaN                          NaN   

                                                                    \
product_id com.gopaktor.credit.v2.3000 com.gopaktor.credit.v2.6000   
cust_id                                                              
2942914                            NaN                         NaN   
3053432                            NaN                         NaN   
3129800                            NaN                         NaN   
3289765                            NaN                         NaN   
3325662                            NaN                         NaN   

                                        \
product_id com.gopaktor.credit.v3.2000   
cust_id                                  
2942914                            NaN   
3053432                            NaN   
3129800                            NaN   
3289765                            NaN   
3325662                            NaN   

                                                    \
product_id com.gopaktor.subscription.v1.premium.1m   
cust_id                                              
2942914                                        NaN   
3053432                                        NaN   
3129800                                        NaN   
3289765                                        NaN   
3325662                                        NaN   

                                                     \
product_id com.gopaktor.subscription.v11.premium.6m   
cust_id                                               
2942914                                         NaN   
3053432                                         NaN   
3129800                                         NaN   
3289765                                         NaN   
3325662                                         NaN   

                                                     \
product_id com.gopaktor.subscription.v12.premium.1m   
cust_id                                               
2942914                                         NaN   
3053432                                         NaN   
3129800                                         NaN   
3289765                                         NaN   
3325662                                        30.0   

                                                     \
product_id com.gopaktor.subscription.v12.premium.3m   
cust_id                                               
2942914                                         NaN   
3053432                                         NaN   
3129800                                         NaN   
3289765                                         NaN   
3325662                                         NaN   

                                                         \
product_id com.gopaktor.subscription.v2.premium.1m.2000   
cust_id                                                   
2942914                                             NaN   
3053432                                             NaN   
3129800                                             NaN   
3289765                                             NaN   
3325662                                             NaN   

                                                         \
product_id com.gopaktor.subscription.v2.premium.1m.3000   
cust_id                                                   
2942914                                             NaN   
3053432                                             NaN   
3129800                         

In [ ]:
type(abc)

In [87]:
mi = abc.columns

In [ ]:
mi

In [88]:
mi.tolist()

[('repurchase_time', 'com.gopaktor.credit.v2.1000'),
 ('repurchase_time', 'com.gopaktor.credit.v2.10000'),
 ('repurchase_time', 'com.gopaktor.credit.v2.3000'),
 ('repurchase_time', 'com.gopaktor.credit.v2.6000'),
 ('repurchase_time', 'com.gopaktor.credit.v3.2000'),
 ('repurchase_time', 'com.gopaktor.subscription.v1.premium.1m'),
 ('repurchase_time', 'com.gopaktor.subscription.v11.premium.6m'),
 ('repurchase_time', 'com.gopaktor.subscription.v12.premium.1m'),
 ('repurchase_time', 'com.gopaktor.subscription.v12.premium.3m'),
 ('repurchase_time', 'com.gopaktor.subscription.v2.premium.1m.2000'),
 ('repurchase_time', 'com.gopaktor.subscription.v2.premium.1m.3000'),
 ('repurchase_time', 'com.gopaktor.subscription.v2.premium.3m'),
 ('repurchase_time', 'com.gopaktor.subscription.v2.premium.3m.2000'),
 ('repurchase_time', 'com.gopaktor.subscription.v2.premium.6m'),
 ('repurchase_time', 'com.gopaktor.subscription.v2.premium.6m.2000'),
 ('repurchase_time', 'com.gopaktor.subscription.v3.premium.3m

In [89]:
ind = pd.Index([e[1] for e in mi.tolist()])

In [90]:
ind

Index([u'com.gopaktor.credit.v2.1000', u'com.gopaktor.credit.v2.10000',
       u'com.gopaktor.credit.v2.3000', u'com.gopaktor.credit.v2.6000',
       u'com.gopaktor.credit.v3.2000',
       u'com.gopaktor.subscription.v1.premium.1m',
       u'com.gopaktor.subscription.v11.premium.6m',
       u'com.gopaktor.subscription.v12.premium.1m',
       u'com.gopaktor.subscription.v12.premium.3m',
       u'com.gopaktor.subscription.v2.premium.1m.2000',
       u'com.gopaktor.subscription.v2.premium.1m.3000',
       u'com.gopaktor.subscription.v2.premium.3m',
       u'com.gopaktor.subscription.v2.premium.3m.2000',
       u'com.gopaktor.subscription.v2.premium.6m',
       u'com.gopaktor.subscription.v2.premium.6m.2000',
       u'com.gopaktor.subscription.v3.premium.3m',
       u'com.gopaktor.subscription.v3.premium.6m',
       u'com.gopaktor.subscription.v4.premium.1m'],
      dtype='object')

In [91]:
abc.columns = ind

In [92]:
abc.head()

,com.gopaktor.credit.v2.1000,com.gopaktor.credit.v2.10000,com.gopaktor.credit.v2.3000,com.gopaktor.credit.v2.6000,com.gopaktor.credit.v3.2000,com.gopaktor.subscription.v1.premium.1m,com.gopaktor.subscription.v11.premium.6m,com.gopaktor.subscription.v12.premium.1m,com.gopaktor.subscription.v12.premium.3m,com.gopaktor.subscription.v2.premium.1m.2000,com.gopaktor.subscription.v2.premium.1m.3000,com.gopaktor.subscription.v2.premium.3m,com.gopaktor.subscription.v2.premium.3m.2000,com.gopaktor.subscription.v2.premium.6m,com.gopaktor.subscription.v2.premium.6m.2000,com.gopaktor.subscription.v3.premium.3m,com.gopaktor.subscription.v3.premium.6m,com.gopaktor.subscription.v4.premium.1m
cust_id,,,,,,,,,,,,,,,,,,
2942914,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3053432,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.111111
3129800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3289765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3325662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
products = pd.DataFrame(abc.columns)

In [ ]:
products

In [94]:
products.rename(columns={0:'product_id'}, inplace=True)

In [ ]:
products

In [198]:
art = abc[products.product_id[products.index]].sum() / abc[products.product_id[products.index]].count()

In [ ]:
type(art)

In [199]:
art = art.to_frame()

In [200]:
art

,0
com.gopaktor.credit.v2.1000,1.586538
com.gopaktor.credit.v2.10000,NaN
com.gopaktor.credit.v2.3000,NaN
com.gopaktor.credit.v2.6000,NaN
com.gopaktor.credit.v3.2000,NaN
com.gopaktor.subscription.v1.premium.1m,30.151834
com.gopaktor.subscription.v11.premium.6m,NaN
com.gopaktor.subscription.v12.premium.1m,30.318182
com.gopaktor.subscription.v12.premium.3m,91.250000
com.gopaktor.subscription.v2.premium.1m.2000,29.507519


In [201]:
art.reset_index(inplace=True)

In [202]:
art

,index,0
0,com.gopaktor.credit.v2.1000,1.586538
1,com.gopaktor.credit.v2.10000,NaN
2,com.gopaktor.credit.v2.3000,NaN
3,com.gopaktor.credit.v2.6000,NaN
4,com.gopaktor.credit.v3.2000,NaN
5,com.gopaktor.subscription.v1.premium.1m,30.151834
6,com.gopaktor.subscription.v11.premium.6m,NaN
7,com.gopaktor.subscription.v12.premium.1m,30.318182
8,com.gopaktor.subscription.v12.premium.3m,91.250000
9,com.gopaktor.subscription.v2.premium.1m.2000,29.507519


In [203]:
art.rename(columns={'index':'product_id'}, inplace=True)

In [204]:
art.rename(columns={0:'average_repurchase_time'}, inplace=True)

In [205]:
art

,product_id,average_repurchase_time
0,com.gopaktor.credit.v2.1000,1.586538
1,com.gopaktor.credit.v2.10000,NaN
2,com.gopaktor.credit.v2.3000,NaN
3,com.gopaktor.credit.v2.6000,NaN
4,com.gopaktor.credit.v3.2000,NaN
5,com.gopaktor.subscription.v1.premium.1m,30.151834
6,com.gopaktor.subscription.v11.premium.6m,NaN
7,com.gopaktor.subscription.v12.premium.1m,30.318182
8,com.gopaktor.subscription.v12.premium.3m,91.250000
9,com.gopaktor.subscription.v2.premium.1m.2000,29.507519


In [178]:
cust_repur_original.head()

,cust_id,product_id,timestamp,count,time,days_int,repurchase_time,xyz
0,2942914,com.gopaktor.subscription.v1.premium.1m,2016-12-24 09:53:35+00:00,NaN,0 days 00:00:00,0,NaN,0
1,3053432,com.gopaktor.subscription.v4.premium.1m,2017-01-26 06:22:00+00:00,10.0,245 days 01:54:56,245,27.1111,245
2,3129800,com.gopaktor.subscription.v1.premium.1m,2016-12-09 07:57:37+00:00,NaN,0 days 00:00:00,0,NaN,0
3,3289765,com.gopaktor.subscription.v2.premium.6m.2000,2016-12-06 11:37:37+00:00,NaN,0 days 00:00:00,0,NaN,0
4,3325662,com.gopaktor.subscription.v12.premium.1m,2016-12-03 04:36:00+00:00,2.0,31 days 01:49:48,31,30,31


In [167]:
cust_repur_original.timestamp.iloc = cust_repur_original.timestamp.iloc.to_datetime()

AttributeError: '_iLocIndexer' object has no attribute 'to_datetime'

In [176]:
type(cust_repur_original.timestamp[0])

pandas.tslib.Timestamp

In [164]:
dt.datetime.today()

datetime.datetime(2017, 3, 26, 14, 25, 36, 710185)

In [187]:
cust_repur_original["timestamp"].dt.date

2790    2016-12-24
3556    2017-01-26
2529    2016-12-09
1308    2016-12-06
2411    2016-12-03
3697    2016-12-30
1949    2016-12-25
2504    2016-12-07
2577    2016-12-12
868     2016-11-15
2       2016-10-01
3264    2017-01-17
1732    2016-11-05
213     2016-10-01
2196    2016-11-22
3       2016-10-01
17      2016-10-02
3550    2017-02-01
556     2016-11-02
58      2016-10-07
1603    2016-11-02
4       2016-10-01
5       2016-10-01
1861    2016-11-09
1602    2016-11-02
2019    2016-11-14
237     2016-10-04
1245    2016-12-06
8       2016-10-01
1285    2016-12-08
           ...    
4386    2017-02-03
4391    2017-02-03
3627    2017-02-03
4390    2017-02-03
3634    2017-02-04
4395    2017-02-04
3638    2017-02-04
4397    2017-02-04
3656    2017-02-04
4398    2017-02-04
3644    2017-02-04
3646    2017-02-04
3648    2017-02-04
3650    2017-02-04
4405    2017-02-04
4410    2017-02-05
4412    2017-02-05
4416    2017-02-05
3658    2017-02-05
3659    2017-02-05
3661    2017-02-05
3663    2017

In [174]:
cust_repur_original["timestamp"] = cust_repur_original["timestamp"].dt.tz_convert('UTC')

In [188]:
cust_repur_original['time_diff'] = dt.date.today() - cust_repur_original["timestamp"].dt.date

In [189]:
cust_repur_original.head()

,cust_id,product_id,timestamp,count,time,time_diff
2790,2942914,com.gopaktor.subscription.v1.premium.1m,2016-12-24 09:53:35+00:00,NaN,0 days 00:00:00,92 days
3556,3053432,com.gopaktor.subscription.v4.premium.1m,2017-01-26 06:22:00+00:00,10.0,245 days 01:54:56,59 days
2529,3129800,com.gopaktor.subscription.v1.premium.1m,2016-12-09 07:57:37+00:00,NaN,0 days 00:00:00,107 days
1308,3289765,com.gopaktor.subscription.v2.premium.6m.2000,2016-12-06 11:37:37+00:00,NaN,0 days 00:00:00,110 days
2411,3325662,com.gopaktor.subscription.v12.premium.1m,2016-12-03 04:36:00+00:00,2.0,31 days 01:49:48,113 days


In [137]:
dt.datetime.now().date()

datetime.date(2017, 3, 26)

In [195]:
cust_repur_original.reset_index(drop=True, inplace=True)

In [196]:
cust_repur_original.head()

,cust_id,product_id,timestamp,count,time,time_diff
0,2942914,com.gopaktor.subscription.v1.premium.1m,2016-12-24 09:53:35+00:00,NaN,0 days 00:00:00,92 days
1,3053432,com.gopaktor.subscription.v4.premium.1m,2017-01-26 06:22:00+00:00,10.0,245 days 01:54:56,59 days
2,3129800,com.gopaktor.subscription.v1.premium.1m,2016-12-09 07:57:37+00:00,NaN,0 days 00:00:00,107 days
3,3289765,com.gopaktor.subscription.v2.premium.6m.2000,2016-12-06 11:37:37+00:00,NaN,0 days 00:00:00,110 days
4,3325662,com.gopaktor.subscription.v12.premium.1m,2016-12-03 04:36:00+00:00,2.0,31 days 01:49:48,113 days


In [206]:
art_dict = art.set_index('product_id')['average_repurchase_time'].to_dict()

In [ ]:
if cust_repur_original["count"] == 0:
    if art_dict.has_key(str(cust_repur_original["product_id"])):
        cust_repur_original.repurchase_time[x] = art_dict[str(cust_repur_original.product_id[x])]